In [1]:
# 安裝yahoo finance套件
!pip install yfinance

In [2]:
import google.generativeai as genai  # Google生成式AI套件Gemini API
import os  # 系統檔，用來操作檔案與文件
import requests  # 用來發HTTP請求，用於網路爬蟲
from bs4 import BeautifulSoup  # 網路爬蟲套件
import numpy as np  # 數值計算
import pandas as pd  # 資料處理
import yfinance as yf  # Yahoo finance 股價資訊
import datetime as dt  # 處理日期時間
import time  # 時間延遲套件，用於控制API呼叫頻率

class StockInfo():
    """
    股票基本資訊爬取類別

    功能：
    1. 從台灣證券交易所爬取所有上市公司的股號、股名、產業別
    2. 提供股號查詢股名的功能

    資料來源：台灣證券交易所 (TWSE) ISIN網站
    """

    # 取得全部股票的股號、股名
    def stock_name(self):
        """
        爬取所有台股的股號、股名、產業別

        Returns:
            DataFrame: 包含「股號」、「股名」、「產業別」三欄的資料表

        技術說明：
        - 使用requests取得網頁內容
        - 使用BeautifulSoup解析HTML
        - 過濾出4碼股號的上市公司（排除ETF等其他商品）
        """
        # 向台灣證券交易所發送請求
        response = requests.get('https://isin.twse.com.tw/isin/C_public.jsp?strMode=2')
        # 使用BeautifulSoup解析HTML內容
        url_data = BeautifulSoup(response.text, 'html.parser')
        # 找出所有的表格列（tr標籤）
        stock_company = url_data.find_all('tr')

        # 資料處理：提取股號、股名、產業別
        # \u3000 是全形空白字元，用來分隔股號和股名
        # 只保留4碼的股號（過濾掉ETF、特別股等）
        data = [
            (row.find_all('td')[0].text.split('\u3000')[0].strip(),
                row.find_all('td')[0].text.split('\u3000')[1],
                row.find_all('td')[4].text.strip())
            for row in stock_company[2:] if len(row.find_all('td')[0].text.split('\u3000')[0].strip()) == 4
        ]
        # 建立DataFrame並設定欄位名稱
        df = pd.DataFrame(data, columns=['股號', '股名', '產業別'])
        return df

    # 取得股票名稱
    def get_stock_name(self, stock_id, name_df):
        """
        根據股號查詢股名

        Args:
            stock_id (str): 股票代號（例如：'2330'）
            name_df (DataFrame): 包含股號和股名的資料表

        Returns:
            str: 股票名稱（例如：'台積電'）
        """
        return name_df.set_index('股號').loc[stock_id, '股名']

In [6]:
class StockAnalysis():
    """
    股票分析類別 - 整合股價資料與Gemini AI分析

    主要功能：
    1. 取得股票價格資料（技術面）
    2. 取得股票基本面資料（營收、EPS）
    3. 使用Gemini AI進行股票分析
    4. 多檔股票比較與推薦

    設計理念：
    - 整合多元數據源（價格、基本面）
    - 利用LLM進行智能分析
    """

    def __init__(self, gemini_api_key):
        """
        初始化股票分析系統

        Args:
            gemini_api_key (str): Google Gemini API金鑰

        初始化步驟：
        1. 設定Gemini API
        2. 建立生成模型實例
        """
        # 設定Gemini API金鑰
        genai.configure(api_key=gemini_api_key)
        """
        1. Gemini 2.5 Pro - 進階思考模型
        2. Gemini 2.5 Flash - 快速且智慧
        3. Gemini 2.5 Flash-Lite - 速度最快的 Flash 模型，可提高成本效益和輸送量。
        """
        # 使用Gemini 2.5 Pro模型（高階推理能力）
        self.model = genai.GenerativeModel('gemini-2.5-pro')
        # 建立股票資訊物件
        self.stock_info = StockInfo()
        # 預先載入所有股票名稱（避免重複爬取）
        self.name_df = self.stock_info.stock_name()


    def stock_price(self, stock_id, days=15):
        """
        取得股票近期價格資料（技術面分析基礎）

        Args:
            stock_id (str): 股票代號（例如：'2330'）
            days (int): 要取得的天數（預設15天）

        Returns:
            dict: 包含日期、收盤價、每日報酬率的字典

        技術說明：
        - 使用yfinance API取得Yahoo Finance資料
        - 計算每日報酬率 = (今日收盤-昨日收盤)/昨日收盤
        - 自動加上.TW後綴（台股代碼格式）
        """

        # 台股代碼需要加上.TW後綴
        stock_id += '.TW'
        # 計算日期區間
        end = dt.date.today() # 今天
        start = end - dt.timedelta(days=days) # 往前推N天

        # 禁用 FutureWarning
        import warnings
        warnings.filterwarnings('ignore', category=FutureWarning)

        # 下載股價資料
        # auto_adjust=True 會自動調整除權息的價格
        df = yf.download(stock_id, start=start, auto_adjust=True)

        # 將欄位名稱改為中文（方便閱讀）
        df.columns = ['開盤價', '最高價', '最低價', '收盤價', '成交量']
        # 建立回傳的資料結構
        data = {
            '日期': df.index.strftime('%Y-%m-%d').tolist(), # 格式化日期
            '收盤價': df['收盤價'].tolist(), # 收盤價列表
            '每日報酬': df['收盤價'].pct_change().tolist() # 報酬率 = 價格變化百分比
        }
        return data


    def stock_fundamental(self, stock_id):
        """
        取得股票基本面資料（價值投資分析基礎）

        Args:
            stock_id (str): 股票代號

        Returns:
            dict: 包含季度日期、營收成長率、EPS、EPS季增率

        基本面指標說明：
        - 營收成長率：季度營收相比上一季的成長百分比
        - EPS (每股盈餘)：公司獲利能力的核心指標
        - EPS季增率：EPS相比上一季的成長百分比

        注意事項：
        - 某些股票可能無基本面資料（新上市或資料不全）
        - 使用try-except確保程式不會因單一股票錯誤而中斷
        """

        stock_id += '.TW'
        stock = yf.Ticker(stock_id)

        try:
            # 計算季度營收成長率
            # pct_change(-1)：與前一期（-1）比較的百分比變化
            # fill_method=None：不填補缺失值
            quarterly_revenue_growth = np.round(
                stock.quarterly_financials.loc['Total Revenue'].pct_change(-1, fill_method=None).dropna().tolist(),
                2
            )
            # 取得季度EPS
            quarterly_eps = np.round(
                stock.quarterly_financials.loc['Basic EPS'].dropna().tolist(),
                2
            )
            # 計算EPS季增率
            quarterly_eps_growth = np.round(
                stock.quarterly_financials.loc['Basic EPS'].pct_change(-1, fill_method=None).dropna().tolist(),
                2
            )
            # 取得日期並格式化
            dates = [date.strftime('%Y-%m-%d') for date in stock.quarterly_financials.columns]
            # 組合資料（確保長度一致）
            data = {
                '季日期': dates[:len(quarterly_revenue_growth)],
                '營收成長率': quarterly_revenue_growth.tolist(),
                'EPS': quarterly_eps.tolist(),
                'EPS 季增率': quarterly_eps_growth.tolist()
            }
        except Exception as e:
            # 如果取得資料失敗，回傳空資料
            print(f"無法取得 {stock_id} 基本面資料: {e}")
            data = {'季日期': [], '營收成長率': [], 'EPS': [], 'EPS 季增率': []}

        return data

    def _get_reply(self, content_msg):
        """
        呼叫Gemini API並處理各種回應狀況（內部方法）

        Args:
            content_msg (str): 要傳送給Gemini的提示詞（prompt）

        Returns:
            str: Gemini的回應文字或錯誤訊息

        錯誤處理機制：
        1. finish_reason = 1 (STOP)：正常完成
        2. finish_reason = 2 (MAX_TOKENS)：超過長度限制
        3. finish_reason = 3 (SAFETY)：觸發安全政策
        4. finish_reason = 4 (RECITATION)：引用問題
        5. HTTP 429：API速率限制（免費版：每分鐘5次，每天25次）
        6. HTTP 503：服務暫時不可用
        """

        try:
            # 呼叫Gemini API生成內容
            response = self.model.generate_content(
                content_msg,
                generation_config=genai.types.GenerationConfig(
                    max_output_tokens=10000,  # 最大輸出token數（避免過長）
                    temperature=1.0,  # 溫度參數（1.0 = 較有創意，0.0 = 較保守）
                )
            )

            # 檢查是否有回應候選
            if not response.candidates:
                return "模型未返回任何回應"

            candidate = response.candidates[0]
            finish_reason = candidate.finish_reason

            # finish_reason是整數值，代表不同的完成狀態：
            # 0 = UNSPECIFIED (未指定)
            # 1 = STOP (正常完成)
            # 2 = MAX_TOKENS (達到token限制)
            # 3 = SAFETY (安全過濾)
            # 4 = RECITATION (引用問題)
            # 5 = OTHER (其他原因)

            # 根據不同的完成原因處理
            if finish_reason == 1:  # STOP - 正常完成
                # 正常完成
                return candidate.content.parts[0].text

            elif finish_reason == 2:  # MAX_TOKENS - 達到長度限制
                try:
                    partial_text = candidate.content.parts[0].text if candidate.content.parts else ""
                    return f"{partial_text}\n\n [回應被截斷，已達到長度限制]"
                except:
                    return "回應超過長度限制。建議：1)增加 max_output_tokens 2)簡化 prompt"

            elif finish_reason == 3:  # SAFETY - 安全政策
                return "模型因安全政策無法提供回應"

            elif finish_reason == 4:  # RECITATION - 引用問題
                return "回應因引用問題被過濾"

            else:
                return f"未知完成原因: {finish_reason}"

        except Exception as e:
            # 處理API錯誤
            error_msg = str(e)
            # 速率限制錯誤（最常見）
            if "429" in error_msg or "RESOURCE_EXHAUSTED" in error_msg:
                return "達到 API 速率限制（免費層：5次/分鐘，25次/天）"
            # 服務暫時不可用
            elif "503" in error_msg:
                return "服務暫時不可用，請稍後重試"
            # 其他錯誤
            else:
                return f"API 錯誤: {error_msg}"


    def stock_gimini_analysis(self, stock_id):
        """
        使用Gemini AI分析單支股票（核心功能）

        Args:
            stock_id (str): 股票代號

        Returns:
            str: Gemini生成的股票分析報告

        分析流程：
        1. 取得股票名稱
        2. 收集技術面資料（價格、報酬率）
        3. 收集基本面資料（營收、EPS）
        4. 將資料摘要後傳送給Gemini
        5. 讓Gemini扮演金融分析師角色，產生專業分析

        Prompt設計要點：
        - 角色設定：明確告訴模型扮演「金融分析師」
        - 數據提供：精簡但完整的關鍵資訊
        - 輸出要求：技術面+基本面+風險評估
        - 語言指定：繁體中文
        """
        # 取得股票名稱
        stock_name = self.stock_info.get_stock_name(stock_id, self.name_df)

        # 收集股價資料
        price_data = self.stock_price(stock_id)
        print('資料收集完畢')
        # 收集基本面資料
        stock_value_data = self.stock_fundamental(stock_id)

        # --- 資料摘要製作 ---
        # 目的：將大量資料濃縮成關鍵指標，避免超過token限制
        summary_parts = []

        # 股價摘要：最新價格 + 兩週漲跌幅
        if price_data and price_data['收盤價']:
            latest = price_data['收盤價'][-1] # 最新收盤價
            first = price_data['收盤價'][0] # 首日收盤價
            change = ((latest - first) / first) * 100 # 計算漲跌百分比
            summary_parts.append(f"股價：{latest:.2f} (兩週 {change:+.2f}%)")

        # 基本面摘要：最新一季的營收成長和EPS
        if stock_value_data and stock_value_data['季日期']:
            q = stock_value_data['季日期'][0] # 季度
            rev = stock_value_data['營收成長率'][0] # 營收成長率
            eps = stock_value_data['EPS'][0] # EPS
            summary_parts.append(f"基本面({q})：營收成長{rev*100:.1f}%, EPS {eps:.2f}")

        # 組合所有摘要
        combined = "；".join(summary_parts)

        # ============ 作業修改區 START ============
        # --- 建構Prompt ---
        # 這是Prompt Engineering的關鍵：如何引導LLM產生專業分析
        """
        'conservative': '''你是保守型財務顧問...
        1. 優先評估下檔風險
        2. 關注股息穩定性
        3. 避免高波動股票...''',

        'aggressive': '''你是成長型投資顧問...
        1. 關注營收成長動能
        2. 尋找突破訊號...''',

        'balanced': '''你是專業金融分析師...
        平衡評估機會與風險...'''
        """
        content_msg = f'''假設你是保守型金融分析師，根據以下股票數據生成投資建議{stock_name}。
資料：{combined}

    請提供詳細的技術面和基本面見解，並考慮風險因素。繁體中文回應。'''
        # ============ 作業修改區 END ============
        # 呼叫Gemini API
        reply = self._get_reply(content_msg)
        return reply


    def stock_gimini_sort(self, message):
        """
        讓Gemini為多檔股票評分並排序

        Args:
            message (dict): 包含多檔股票分析報告的字典

        Returns:
            str: Gemini生成的評分排序結果

        評分標準：
        - 50分為基準
        - 正面因素加分（例如：營收成長、技術面轉強）
        - 負面因素扣分（例如：虧損、下跌趨勢）
        - 綜合技術面+基本面+市場環境

        應用場景：投資組合篩選、資金配置決策
        """

        content_msg = f'''你是專業股票分析師，根據趨勢報告評分(0-100)。
50分為基準，正面消息加分，負面消息扣分。
排序所有股票。{str(message)}(繁體中文)'''
        reply = self._get_reply(content_msg)
        return reply


    def stock_gimini_choice(self, message):
        """
        讓Gemini從多檔股票中選出最佳投資標的

        Args:
            message (dict): 包含多檔股票分析報告的字典

        Returns:
            str: Gemini選出的最佳股票及理由

        決策邏輯：
        - 即使所有選項都不理想，也要選出相對最佳的
        - 需要說明選擇理由（風險、報酬、時機等）
        - 模擬真實的投資決策情境

        教學價值：
        - 理解LLM如何進行多維度評估
        - 學習投資決策的思考框架
        """

        content_msg = f'''你是專業證券分析師，從趨勢報告中選出最適合投資的一檔股票。
即使都不理想也要選一檔，說明理由。{str(message)}(繁體中文)'''
        reply = self._get_reply(content_msg)
        return reply

In [7]:
# 【重要】請修改為你自己的Gemini API Key
# ============ 作業修改區 START ==========
gemini_api_key = 'AIzaSyB4R8qxPAXJ3pdBusQdOpanFEOgYyD123Lo'
# ============ 作業修改區 END ============
# 建立股票分析物件
stock_analysis = StockAnalysis(gemini_api_key)

In [8]:
# ============ 作業修改區 START ==========
# 改成想分析的股票代號
reply = stock_analysis.stock_gimini_analysis('2382')
# ============ 作業修改區 END ============
print(reply)

[*********************100%***********************]  1 of 1 completed


資料收集完畢
好的，身為一名保守型金融分析師，我將根據您提供的數據，並結合市場現況，為您提供一份關於廣達（2382）的詳細投資建議。

我的分析核心將圍繞**風險控管**與**價值評估**，這是一個保守型投資者最應關注的兩個面向。

---

### **投資建議總結**

**結論：建議「謹慎觀望」，不宜在當前價位追高買入。**

對於**空手投資者**，建議耐心等待股價出現更大幅度的回檔，或基本面數據出現更強勁的證實。對於**已持股者**，可考慮在目前的高檔位置適度減碼，鎖定部分獲利，或設定嚴格的移動停利點。

主要理由如下：

1.  **高估值風險：** 目前的股價已大幅反映了市場對其 AI 伺服器業務的樂觀預期，估值處於歷史高位，安全邊際不足。
2.  **基本面數據疑慮：** 您提供的未來營收成長預測為負值，這與市場火熱的 AI 題材形成強烈對比，顯示其核心業務（如筆電）可能面臨壓力，或 AI 業務的貢獻尚不足以完全彌補其他業務的衰退。
3.  **技術面警訊：** 股價在高檔出現盤整且小幅回落，顯示上攻動能有所趨緩，短期內需提防獲利了結賣壓。

---

### **詳細分析**

#### **一、 技術面見解 (Technical Analysis)**

從技術分析角度來看，保守型投資者應關注趨勢的穩定性與潛在的轉折風險。

*   **當前價位與趨勢：**
    *   股價 278.00 元位於歷史相對高檔區。雖然過去一年受惠於 AI 題材，股價呈現非常強勁的長多趨勢，但這也意味著累積了大量的獲利賣壓。
    *   「兩週 -3.47%」的數據表明，股價在創高後出現了**短期回檔**或**高檔整理**的跡象。這本身不是壞事，但對於保守型投資者而言，這是一個信號，表明市場多空力量開始出現分歧，追價風險正在升高。

*   **支撐與壓力：**
    *   **壓力：** 前波高點將形成心理壓力區，若無法帶量突破，則可能形成頭部（M頭或圓弧頂）型態。
    *   **支撐：** 下方的關鍵支撐區可能落在月線（約20日均線）或季線（約60日均線）。一旦股價跌破月線且無法迅速站回，則短期趨勢可能轉弱；若進一步跌破季線，則中期趨勢可能面臨反轉風險。

*   **量價關係：**
    *   需密切觀察股價下跌時是否伴隨成交量放大。若出

In [9]:
# 建立股票清單
# ============ 作業修改區 START ============
# 請修改為你想分析的股票清單（建議3-5檔，避免超過API限制）
# 台股股票參考：
# 2330 台積電, 2317 鴻海, 2454 聯發科, 2412 中華電, 2881 富邦金
# 2382 廣達, 2303 聯電, 3711 日月光投控, 2891 中信金, 2886 兆豐金
stock_list = ['2317', '2330', '2454']
# ============ 作業修改區 END ============

# 設定儲存路徑
today_time = dt.date.today().strftime('%Y%m%d')
path = './StockGemini/TrendReport/'
os.makedirs(path, exist_ok=True)

# 建立多檔股票的趨勢報告並儲存
content = {}

print(f"\n{'='*70}")
print(f"開始分析 {len(stock_list)} 檔股票")
print(f"{'='*70}\n")

# 批次處理每一檔股票
for i, stock in enumerate(stock_list, 1):
    print(f"[{i}/{len(stock_list)}] 處理股票: {stock}")

    file_path = f"{path}trend_{stock}_{today_time}.txt"

    if not os.path.exists(file_path):
        try:
            # 分析股票
            print(f"   正在分析...")
            analysis = stock_analysis.stock_gimini_analysis(stock_id=stock)

            # 儲存結果
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(analysis)

            content[stock] = analysis
            print(f"   分析完成並已儲存\n")

            # 速率限制控制
            if i < len(stock_list):
                wait_time = 12
                print(f"   等待 {wait_time} 秒以符合 API 速率限制...\n")
                time.sleep(wait_time)

        except Exception as e:
            error_msg = f"錯誤: {str(e)}"
            print(f"   {error_msg}\n")
            content[stock] = error_msg

    else:
        # 讀取已存在的分析
        with open(file_path, "r", encoding="utf-8") as f:
            content[stock] = f.read()
        print(f"   從快取讀取\n")

print(f"{'='*70}")
print("所有股票分析完成！")
print(f"{'='*70}\n")

print("\n" + "="*70)
print("分析結果")
print("="*70 + "\n")

for stock_id, analysis in content.items():
    stock_name = stock_analysis.stock_info.get_stock_name(stock_id, stock_analysis.name_df)

    print("="*70)
    print(f"【{stock_id} - {stock_name}】")
    print("="*70)
    print(analysis)
    print("\n")


開始分析 3 檔股票

[1/3] 處理股票: 2317
   正在分析...


[*********************100%***********************]  1 of 1 completed


資料收集完畢
   分析完成並已儲存

   等待 12 秒以符合 API 速率限制...

[2/3] 處理股票: 2330
   正在分析...


[*********************100%***********************]  1 of 1 completed


資料收集完畢
   分析完成並已儲存

   等待 12 秒以符合 API 速率限制...

[3/3] 處理股票: 2454
   正在分析...


[*********************100%***********************]  1 of 1 completed


資料收集完畢
   分析完成並已儲存

所有股票分析完成！


分析結果

【2317 - 鴻海】
好的，作為一名保守型金融分析師，我將根據您提供的數據，結合市場現況，為您提供一份關於鴻海 (2317) 的詳細投資建議。

---

### **鴻海 (2317) 投資分析報告 (保守型觀點)**

**報告日期：** 2024年6月

**核心摘要：**
鴻海正處於從傳統電子代工 (EMS) 轉型為 AI 伺服器與電動車解決方案提供商的關鍵時期。其基本面前景因 AI 伺服器業務而顯著改善，長期趨勢看好。然而，近期股價經歷一輪大漲後出現顯著回檔，短期技術面呈現壓力。

對於保守型投資者而言，當前的策略應是**「短期謹慎觀望，長期分批佈局」**。不建議在技術指標尚未穩定時追高或一次性投入大量資金，而應利用價格修正的機會，為長期投資建立成本優勢。

---

### **一、 技術面分析**

1.  **短期趨勢與動能：**
    *   **股價表現：** 目前股價 225.00 元，近兩週下跌 8.16%，顯示自前波高點以來，賣壓明顯湧現，股價進入短期修正階段。
    *   **價量關係：** 需要觀察此次下跌是否伴隨巨大成交量。若為「價跌量增」，表示賣方力道強勁，修正可能持續；若為「價跌量縮」，則意味著賣壓趨緩，籌碼趨於穩定，是較好的止跌訊號。
    *   **技術指標：** 經過先前的大漲，各項指標（如 RSI、KD）普遍從「超買區」或「高檔鈍化區」回落，這是技術性修正的正常現象。保守操作者應等待指標落至中性區間或出現黃金交叉等止跌訊號後，再行考慮。

2.  **支撐與壓力：**
    *   **初步支撐：** 可觀察月線（20MA）或季線（60MA）的支撐力道。這些移動平均線是市場參與者的平均成本，通常具有重要的心理與實質支撐作用。跌破月線後，季線將是下一道關鍵防線。
    *   **關鍵支撐區：** 前一波整理平台的頸線位置或整數關卡（如 200 元）將是強力的支撐區。若股價能在此區域獲得支撐並橫盤整理，將是技術面回穩的重要跡象。
    *   **上方壓力：** 前波高點將成為短期內的主要壓力區，需要有重大利好消息或強勁的買盤動能才有機會突破。

**技術面結論：**
目前股價處於**短期空頭修正格局**，動能偏弱。保守投資者不應在

In [10]:
reply = stock_analysis.stock_gimini_choice(str(content))
print(reply)

好的，作為一名專業證券分析師，我將詳細審閱您提供的三份趨勢報告，並基於這些資訊，為您選出最值得投資的一檔股票，並闡述我的決策邏輯。

---

### **最終投資選擇**

經過綜合評估三份趨勢報告，我會選擇 **2317 鴻海** 作為最適合投資的標的。

### **核心決策理由**

我的選擇基於一個核心原則：**在合理的價格，買入具備清晰成長前景的公司。** 鴻海在這三者中，提供了最佳的「成長性」與「價值性」組合，儘管其短期技術面正在修正，但這反而為長期投資者創造了潛在的切入機會。

以下是我對三檔股票的詳細比較與篩選過程：

---

### **1. 為什麼選擇 2317 鴻海 (Hon Hai)**

鴻海的報告揭示了一個最為吸引人的投資劇本：**價值重估 (Re-rating) + 強勁成長**。

*   **基本面最強勁：** 報告預估其營收將有 **15.0% 的年增長**。對於一家市值如此龐大的公司而言，這是極其強勁的成長動能，主要來自市場最熱門的 AI 伺服器業務。這表明公司的轉型策略正在兌現為實質的營收。
*   **估值最具吸引力：** 根據報告推算，其預估本益比 (Forward P/E) 約 **13.5 倍**。相較於其他 AI 概念股，這個估值顯然偏低。這代表市場尚未完全反映其從傳統代工廠轉型為 AI 硬體巨擘的潛在價值，為股價提供了向上擴張的空間。
*   **風險可控：** 報告中提到的短期技術面回檔 (-8.16%)，我視其為「健康的修正」而非「趨勢反轉」。在大漲之後進行整理，有助於籌碼沉澱。對於價值投資者而言，股價的修正正是**創造安全邊際 (Margin of Safety)** 的良機。

**結論：鴻海提供了清晰的成長故事（AI 伺服器）、強勁的財務預期（營收+15%），以及相對低廉的估值（13.5倍本益比），使其成為三者中風險回報比最佳的選擇。**

---

### **2. 為什麼不優先選擇 2330 台積電 (TSMC)**

台積電無疑是一家偉大的公司，但根據報告分析，現在的它是一個**「好的公司，昂貴的價格」**。

*   **核心問題：估值過高 (Valuation Risk)**。報告明確指出，其預估本益比**高達 92.77 倍**。這是一個極端的數字，意味著股價已經嚴重透支了未來的樂觀預期。

In [11]:
reply = stock_analysis.stock_gimini_sort(str(content))
print(reply)

好的，這是一份基於您提供的三份趨勢報告所做的專業評分與排序。

我將依據每份報告中揭示的**基本面前景**、**估值合理性**、**技術面趨勢**以及**分析師的最終建議傾向**，進行綜合評分。

---

### **股票趨勢報告評分與排序**

| 排序 | 股票代號 | 股票名稱 | 趨勢評分 (0-100) | 核心觀點                                                                                                                                                 |
| :--- | :------- | :------- | :--------------- | :------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **1**  | **2317**   | **鴻海**   | **80** / 100     | **正面 (長期看多，短期修正為買點)**<br>基本面強勁，AI伺服器題材提供明確成長動能，估值相對同業偏低。雖然短期技術面因漲多而修正，但報告傾向視為長期佈局的機會點。 |
| **2**  | **2330**   | **台積電** | **25** / 100     | **負面 (公司好，但價格過高)**<br>公司體質優異，護城河深厚，但預估本益比超過90倍，估值風險極高，已嚴重壓縮未來回報空間。報告明確指出「價格昂貴」，建議空手者耐心等待。 |
| **3**  | **2454**   | **聯發科** | **10** / 100     | **負面 (基本面與技術面雙重警訊)**<br>核心風險在於未來營收預測出現**負成長**，這是嚴重的基本面警訊。同時技術面處於下降趨勢，報告態度最為悲觀，甚至建議既有持股者考慮賣出。 |

---

### **評分詳細解析**

#### **第一名：2317 鴻海 (評分: 8